In [1]:
import torch
import torch.nn as nn
import string
import numpy as np
import random
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from IPython.core.debugger import set_trace

In [2]:
def get_alphabets_random(window, shift, text=None, reverse=False):
    if text is not None:
        shift = shift % len(text)
        shifted_text = text[shift:] + text[:shift]
        return f'Input: {text}\nShifted: {shifted_text}'
    
    alphabets = string.ascii_lowercase
    rand_indices = [random.randint(0, 25) for _ in range(window)]
    
    text = ''.join(list(map(lambda x: alphabets[x], rand_indices)))
    
    if reverse:
        reversed_text = ''.join(list(reversed(list(text))))
        
    shift = shift % len(text)
    shifted_text = text[shift:] + text[:shift]
    
    return text, shifted_text

get_alphabets_random(10, -2, reverse=False)

('zpoyqqytua', 'uazpoyqqyt')

In [3]:
def generate_training_data(window, shift, num_examples):
    x = list(); y = list()
    
    for _ in range(num_examples):
        text, shifted = get_alphabets_random(window, shift, reverse=False)
        x.append(text); y.append(shifted)
        
    random_id = random.randint(0, len(x))
    print(x[random_id], y[random_id])
    
    return x, y

In [4]:
def create_vocabulary():
    unique_characters = list(string.ascii_lowercase)
    char_to_id = {char:i for i, char in enumerate(unique_characters)}
    id_to_char = {i: char for i, char in enumerate(unique_characters)}
    
    features_dim = len(unique_characters)
    print(f'Number of features is {features_dim}')
    return char_to_id, id_to_char, features_dim

In [5]:
encode_text = lambda x: [char_to_id[_] for _ in x]
decode_text = lambda x_: [id_to_char[_] for _ in x_]

In [6]:
def encode_data(x, y, features_dim):
    encoded_x = list(map(lambda x_: encode_text(x_), x))
    encoded_y = list(map(lambda x_: encode_text(x_), y))
    
    one_hot_x = F.one_hot(torch.tensor(encoded_x), num_classes=features_dim)
    one_hot_y = F.one_hot(torch.tensor(encoded_y), num_classes=features_dim)
    
    one_hot_y = one_hot_y.permute(1, 0, 2)
    
    return one_hot_x.to(torch.float32), one_hot_y.to(torch.float32)

In [7]:
x, y = generate_training_data(10, -2, 10000)
char_to_id, id_to_char, features = create_vocabulary()
one_hot_x, one_hot_y = encode_data(x, y, features)
print(one_hot_x.dtype, one_hot_y.dtype)

val_x, val_y = generate_training_data(10, -2, 1000)
val_x, val_y = encode_data(val_x, val_y, features)
print(val_x.shape, val_y.shape)

bqiexttorv rvbqiextto
Number of features is 26
torch.float32 torch.float32
ouuzfrrunh nhouuzfrru
torch.Size([1000, 10, 26]) torch.Size([10, 1000, 26])


In [8]:
decode_text(torch.argmax(one_hot_x[0], dim=-1).tolist()), decode_text(torch.argmax(one_hot_y[:, 0, :], dim=-1).tolist())

(['h', 'u', 'l', 'z', 'f', 'o', 'e', 'r', 'h', 'e'],
 ['h', 'e', 'h', 'u', 'l', 'z', 'f', 'o', 'e', 'r'])

In [9]:
class CustomData(Dataset):
    def __init__(self, x_data, y_data):
        super().__init__()
        self.x_data = x_data
        self.y_data = y_data
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[:, idx, :]
    
dataset = CustomData(one_hot_x, one_hot_y)
val_dataset = CustomData(val_x, val_y)

train_loader = DataLoader(dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=True)

In [20]:
class AttentionModel(nn.Module):
    def __init__(self, encoder_dim, decoder_dim, input_features_dim, output_features_dim, timesteps, num_training):
        super().__init__()
        self.timesteps = timesteps
        self.input_features_dim = input_features_dim
        
        self.densor1 = nn.Linear(decoder_dim+(2*encoder_dim), 10)
        self.tanh = nn.Tanh()
        self.densor2 = nn.Linear(10, 1)
        self.softmax = nn.Softmax(dim=1)
        
        self.pre_attention_lstm = nn.LSTM(input_size=input_features_dim, hidden_size=encoder_dim, batch_first=True, bidirectional=True)
        self.post_attention_lstm = nn.LSTM(input_size=encoder_dim*2, hidden_size=decoder_dim, batch_first=True)
        self.output_layer = nn.Linear(decoder_dim, output_features_dim)
        
        
    def forward(self, x, s0, c0):
        # x-> (B, timesteps, features) -> (128, 10, 26)
        # s0, c0 -> (1, B, decoder_dim) -> (1, 128, 16)
        #set_trace()
        self.s = s0; self.c = c0
        encoder_hidden_states, (hn, cn) = self.pre_attention_lstm(x) #hidden_states: (B, timesteps, encoder_dim*2) -> (128, 10, 16)
        outputs = list()
        
        for t in range(self.timesteps):
            context = self.one_step_attention(encoder_hidden_states) # (B, timesteps, 2*encoder_dim)
            _, (self.s, self.c )= self.post_attention_lstm(context, (self.s, self.c)) # _ -> (B, timesteps, decoder_dim)
            output = self.output_layer(self.s) # (1, B, output_features_dim) -> (1, 128, 26)
            outputs.append(output) # (timesteps, B, output_features_dim) -> (10, 128, 26)
        
        return outputs
    
    def one_step_attention(self, encoder_hidden_states):
        # encoder_Hidden_states -> (B, timesteps, 2*encoder_dim) -> (128, 10, 16)
        # self.s -> (1, B, decoder_dim)
        # 1. first the hidden state for decoder must be repeated to match the hidden states of encoder
        # self.s -> (timesteps, B, decoder_dim) -> permute -> (B, timesteps, decoder_dim)
        # 2. then concatenate the hidden state for decoder and the hidden state for encoder -> (B, timesteps, 2*encoder_dim + decoder_dim)
        # pass it through the first dense layer
        # pass it through the second dense layer
        # use softmax to decide which hidden state of encoder is the most important
        # use dot product to find the important hidden state of encoder and feed it as input to the decoder
        hidden_decoder = self.s.repeat(10, 1, 1).permute(1, 0, 2) # (B, timesteps, decoder_dim) -> (128, 10, 16)
        concat = torch.concatenate([encoder_hidden_states, hidden_decoder], dim=-1) # (B, timesteps, 2*encoder_dim + decoder_dim) -> (128, 10, 32)
        e = self.tanh(self.densor1(concat)) # (B, timesteps, 10) -> (128, 10, 10)
        energies = self.softmax(self.densor2(e)) # (B, timesteps, 1) -> (128, 10, 1)
        # let's if without permute in next step if the code converges
        energies = energies.repeat(1, 1, 10).permute(0, 2, 1) # (B, timesteps, 10) -> (B, 10, 10)
        context = torch.bmm(energies, encoder_hidden_states) # (B, timesteps, timesteps) @ (B, timesteps, 2*encoder_dim) -> (B, timesteps, 2*encoder_dim)
        
        return context
        
    def predict(self, x):
        assert len(x) == self.timesteps
        encoded = encode_text(x)
        one_hot = F.one_hot(torch.tensor(encoded), num_classes=self.input_features_dim)
        s0 = torch.zeros(1, 1, decoder_dim); c0 = torch.zeros(1, 1, decoder_dim)
        one_hot = one_hot.unsqueeze(0)
        pred = self.forward(one_hot.to(torch.float32), s0, c0)
        pred = ''.join(decode_text([torch.argmax(t, dim=-1).tolist()[0][0] for t in pred]))
        return pred

In [21]:
def calculate_loss(y_true, y_pred):
    ce = nn.CrossEntropyLoss()
    total = 0
    for target, logit in zip(list(y_true), list(y_pred)):
        loss = ce(logit[0], target)
        total += loss
        
    return total

In [23]:
encoder_dim = 8
decoder_dim = 16
input_features_dim = features
output_features_dim = features
timesteps = 10
num_training = 10000
epochs = 40

s0, c0 = torch.zeros(1, num_training, decoder_dim), torch.zeros(1, num_training, decoder_dim)

am = AttentionModel(encoder_dim, decoder_dim, input_features_dim, output_features_dim, timesteps, num_training)
opt = optim.Adam(am.parameters())

for epoch in range(epochs):
    am.train()
    train_loss = list()
    for i, (inputs, target) in enumerate(train_loader):
        #set_trace()
        target = target.permute(1, 0, 2)
        batch_size = inputs.size(0)
        s0 = torch.zeros(1, batch_size, decoder_dim); c0 = torch.zeros(1, batch_size, decoder_dim)
        outputs = am(inputs, s0, c0)
        
        opt.zero_grad()
        
        total_loss = calculate_loss(target, outputs)
        train_loss.append(total_loss)
        
        total_loss.backward()
        opt.step()
        
    print(f'Epoch {epoch}:: Train Loss {torch.mean(torch.tensor(train_loss))}')
    
    am.eval()
    with torch.no_grad():
        s0 = torch.zeros(1, 1000, decoder_dim); c0 = torch.zeros(1, 1000, decoder_dim)
        out = am(val_x, s0, c0)
        
        val_loss = calculate_loss(val_y, out)
        
        print(f'Epoch {epoch}:: Val Loss {val_loss}')
        test = 'monojitabc'
        pred = am.predict(test)
        print(f'Input {test} --> Output {pred}')
        print()
    

Epoch 0:: Train Loss 32.61753845214844
Epoch 0:: Val Loss 32.46696090698242
Input monojitabc --> Output iiiiiiiiii

Epoch 1:: Train Loss 31.962995529174805
Epoch 1:: Val Loss 31.577430725097656
Input monojitabc --> Output oooooooooo

Epoch 2:: Train Loss 31.417795181274414
Epoch 2:: Val Loss 31.297893524169922
Input monojitabc --> Output oooooooooo

Epoch 3:: Train Loss 31.16803741455078
Epoch 3:: Val Loss 31.033721923828125
Input monojitabc --> Output oooooooooo

Epoch 4:: Train Loss 30.827407836914062
Epoch 4:: Val Loss 30.787288665771484
Input monojitabc --> Output oooooooooo

Epoch 5:: Train Loss 30.51252555847168
Epoch 5:: Val Loss 30.369169235229492
Input monojitabc --> Output oooooooooo

Epoch 6:: Train Loss 30.182771682739258
Epoch 6:: Val Loss 30.01956558227539
Input monojitabc --> Output oooooooooo

Epoch 7:: Train Loss 29.752531051635742
Epoch 7:: Val Loss 29.481632232666016
Input monojitabc --> Output cooooooooo

Epoch 8:: Train Loss 29.17110824584961
Epoch 8:: Val Loss 28.

In [25]:
test = 'thisabcdifa'
pred = am.predict(test)
pred

AssertionError: 

In [14]:
test_x, test_y = encode_data(x, y, features)

In [15]:
calculate_loss(one_hot_y, outputs)

ValueError: Expected input batch_size (16) to match target batch_size (10000).

In [ ]:
one_hot_y[:, 10, :]

In [ ]:
torch.argmax(one_hot_y[0], dim=-1).shape

In [ ]:
ce = nn.CrossEntropyLoss()
target_label = torch.argmax(one_hot_y[0], dim=-1)
print(outputs[0].shape, target_label.shape)
ce(outputs[0][0], target_label), ce(outputs[0][0], one_hot_y[0])

In [ ]:
out2[0].repeat(1, 10).shape